# Fine Tuning Sentiment Analysis

In [31]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, TrainingArguments, AutoModelForSequenceClassification, Trainer, pipeline
from torchinfo import summary
import json
import evaluate
import numpy as np

In [2]:
# load the glue dataset specialized in sentiment analysis sst2
raw_datasets = load_dataset('glue', 'sst2') 

# try with "amazon_polarity" dataset

In [3]:
raw_datasets
# get a list of the methods available on a dataset
# dir(raw_datasets['train'])
# get the path of the documentation
# type(raw_datasets['train'])


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

we can index the data set with a range of indices, just like a list.

We can also get the name of the classes Ids, with the features method

In [4]:
print(raw_datasets['train'][10:13])
print(raw_datasets['train'].features)

{'sentence': ['goes to absurd lengths ', "for those moviegoers who complain that ` they do n't make movies like they used to anymore ", "the part where nothing 's happening , "], 'label': [0, 0, 0], 'idx': [10, 11, 12]}
{'sentence': Value(dtype='string', id=None), 'label': ClassLabel(names=['negative', 'positive'], id=None), 'idx': Value(dtype='int32', id=None)}


### Tokenizer

In [5]:
# checkpoint = 'bert-base-uncased'
checkpoint = 'distilbert-base-uncased' # trains faster
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [6]:
tokenized_sentences = tokenizer(raw_datasets['train'][0:3]['sentence'], truncation=True)
from pprint import pprint
pprint(tokenized_sentences)

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 'input_ids': [[101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102],
               [101,
                3397,
                2053,
                15966,
                1010,
                2069,
                4450,
                2098,
                18201,
                2015,
                102],
               [101,
                2008,
                7459,
                2049,
                3494,
                1998,
                10639,
                2015,
                2242,
                2738,
                3376,
                2055,
                2529,
                3267,
                102]]}


In [7]:
def tokenize_fn(batch):
  return tokenizer(batch['sentence'], truncation=True)

In [8]:
tokenized_datasets = raw_datasets.map(tokenize_fn, batched=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Map: 100%|██████████| 67349/67349 [00:00<00:00, 69985.77 examples/s]


### Training Arguments

In [12]:
training_args = TrainingArguments(
  'sentiment_analysis_trainer',
  evaluation_strategy='epoch',
  save_strategy='epoch',
  num_train_epochs=1,
)

### Model

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors: 100%|██████████| 268M/268M [00:40<00:00, 6.66MB/s] 
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
type(model)

transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification

In [17]:
# with torchinfo summary
summary(model)

Layer (type:depth-idx)                                  Param #
DistilBertForSequenceClassification                     --
├─DistilBertModel: 1-1                                  --
│    └─Embeddings: 2-1                                  --
│    │    └─Embedding: 3-1                              23,440,896
│    │    └─Embedding: 3-2                              393,216
│    │    └─LayerNorm: 3-3                              1,536
│    │    └─Dropout: 3-4                                --
│    └─Transformer: 2-2                                 --
│    │    └─ModuleList: 3-5                             42,527,232
├─Linear: 1-2                                           590,592
├─Linear: 1-3                                           1,538
├─Dropout: 1-4                                          --
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0

Let's save the un-trained parameters in a list so that we can compare after training

In [18]:
params_before = []
for name, p in model.named_parameters():
  params_before.append(p.detach().cpu().numpy())

### Metrics

In [21]:
metric = load_metric('glue', 'sst2')

/var/folders/cv/rf4bgk9d1sqgbjbk5ydbt4740000gn/T/ipykernel_51427/2092214033.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', 'sst2')


In [24]:
glue_metric = evaluate.load('glue', 'sst2')

In [25]:
print(metric.compute(predictions=[1, 0, 1], references=[1, 0, 0]))
print(glue_metric.compute(predictions=[1, 0, 1], references=[1, 0, 0]))

{'accuracy': 0.6666666666666666}
{'accuracy': 0.6666666666666666}


we have to create this function to match the Trainer API

In [26]:
def compute_metrics(logits_and_labels):
  logits, labels = logits_and_labels
  predictions = np.argmax(logits, axis=-1)
  return glue_metric.compute(predictions=predictions, references=labels)

### Trainer

In [27]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [28]:
trainer.train()

  6%|▌         | 501/8419 [01:04<13:25,  9.84it/s] 

{'loss': 0.4331, 'learning_rate': 4.7030526190759e-05, 'epoch': 0.06}


 12%|█▏        | 1002/8419 [01:53<11:26, 10.80it/s]

{'loss': 0.3376, 'learning_rate': 4.4061052381518e-05, 'epoch': 0.12}


 18%|█▊        | 1501/8419 [02:42<10:47, 10.69it/s]

...

                                                   
100%|██████████| 8419/8419 [14:47<00:00,  4.39it/s]

{'eval_loss': 0.35671108961105347, 'eval_accuracy': 0.9048165137614679, 'eval_runtime': 4.4444, 'eval_samples_per_second': 196.203, 'eval_steps_per_second': 24.525, 'epoch': 1.0}


100%|██████████| 8419/8419 [14:49<00:00,  9.47it/s]

{'train_runtime': 889.0206, 'train_samples_per_second': 75.756, 'train_steps_per_second': 9.47, 'train_loss': 0.2665088724702055, 'epoch': 1.0}


TrainOutput(global_step=8419, training_loss=0.2665088724702055, metrics={'train_runtime': 889.0206, 'train_samples_per_second': 75.756, 'train_steps_per_second': 9.47, 'train_loss': 0.2665088724702055, 'epoch': 1.0})

In [29]:
trainer.save_model('fine_tuned_sentiment_analysis_model')

In [32]:
newmodel = pipeline('text-classification', model='fine_tuned_sentiment_analysis_model', device='mps')

In [33]:
print(newmodel('This movie is great!'))
print(newmodel('This movie is definitely not the best one I have seen lately!'))

[{'label': 'LABEL_1', 'score': 0.9988316893577576}]
[{'label': 'LABEL_0', 'score': 0.9944121241569519}]


Modify the config.json file of the model so that we give names to labels

In [34]:
config_path = 'fine_tuned_sentiment_analysis_model/config.json'
with open(config_path) as f:
  j = json.load(f)

j['id2label'] = {0: 'negative', 1: 'positive'}

with open(config_path, 'w') as f:
  json.dump(j, f, indent=2)

Reload the model

In [36]:
newmodel = pipeline('text-classification', model='fine_tuned_sentiment_analysis_model', device='mps')

In [37]:
newmodel('This movie is great!')

[{'label': 'positive', 'score': 0.9988316893577576}]

### Compare the model's parameters before and after training

In [39]:
params_after = []
for name, p in model.named_parameters():
  params_after.append(p.detach().cpu().numpy())

In [40]:
# verify that the params have changed
for p1, p2 in zip(params_before, params_after):
  print(np.sum(np.abs(p1 - p2)))

13300.65
89.88836
1.7955394
1.1239388
1297.5863
1.7593204
1288.6768
0.003140492
1196.1934
1.0895162
1132.2045
0.84817284
1.6469915
0.8338374
4953.178
5.767476
4541.567
0.69504136
1.5693657
0.7349802
1282.287
1.421695
1277.6827
0.0033160353
1129.1239
0.8821959
1077.76
0.7430595
1.524674
0.73176754
4924.571
5.45126
4496.398
0.7148503
1.5088928
0.80420995
1275.1122
1.504866
1299.787
0.0026822262
1142.9791
0.8354318
1119.2639
0.71764046
1.5220361
0.77335036
4983.0083
5.5937324
4477.8286
0.74410963
1.3862755
0.79971766
1310.8688
1.5094472
1323.6318
0.003470317
1178.9724
0.8257409
1130.1505
0.8058703
1.4013195
0.78627086
4879.233
5.4010105
4265.6304
0.7887438
1.3308091
0.8252969
1200.4796
1.6016853
1191.1804
0.002376105
998.4962
0.78034025
1006.103
0.90565497
1.4091291
0.94643724
4442.6836
5.0295005
3520.7502
0.7151392
1.3262099
0.7596569
1107.5911
1.1820103
1130.9719
0.001162499
920.14825
0.88509226
900.5255
1.1747687
1.3468661
1.1817567
3555.3335
4.628983
3045.2712
0.8681102
1.2758484
0.67